In [ ]:
! pip install --upgrade tensorflow

In [ ]:
! pip install --upgrade tensorflow-hub

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split

In [ ]:
path='./Resources/data.csv'
df=pd.read_csv(path)
df.head()

In [ ]:
df_new=df.fillna(" ")

In [ ]:
y=df_new['Label'].astype(float) 
X=df_new['Headline'] 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [ ]:
embedding = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(X_train)

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(60, activation='relu'))
model.add(tf.keras.layers.Dense(25, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_test,y_test))

In [ ]:
print(model.evaluate(X_test,y_test))

In [ ]:
model.save("savemodel")

In [ ]:
from sklearn.metrics import classification_report
prediction = model.predict(X_test)
prediction = np.array([1 if row>=0.5 else 0 for row in prediction])
print(classification_report(y_test, prediction))

In [ ]:
! pip install mlflow

In [ ]:
import mlflow
import mlflow.tensorflow

mlflow.tensorflow.autolog()

In [ ]:
loaded_model = tf.keras.models.load_model('savemodel')

In [ ]:
loaded_model.summary()

In [ ]:
mlflow.keras.save_model(model, "keras_model")

In [ ]:
! pip install keras

In [ ]:
loaded_keras_model = mlflow.keras.load_model("keras_model", custom_objects={'KerasLayer':hub.KerasLayer})

In [ ]:
X_test[:2].to_json(orient="split")

In [ ]:
tf.compat.v1.keras.experimental.export_saved_model(model, 'model.h5')

In [ ]:
loaded_model = tf.compat.v1.keras.experimental.load_from_saved_model("model.h5", custom_objects={'KerasLayer':hub.KerasLayer})